# Variational autoencoder
This notebook will investiga

In [1]:
%matplotlib
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from variational_autoencoder import VariationalAutoencoder
from autoencoder import Autoencoder
from cont_bern_loss import cont_bern_loss

# For dimension reduction
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# for visualization and evaluation
from latent_plane_mosaic import LatentPlaneMosaic
from latent_interpolation_mosaic import LatentInterpolationMosaic
from sample_scatter_gui import SampleScatterGUI

Using matplotlib backend: Qt5Agg


In [10]:
# Global settings
model_path = ["models\\ae", "models\\vae_10", "models\\vae_100" , "models\\vae_1000"]
model_type = ["ae", "vae", "vae", "vae"]
num_samples = 10000
num_models = len(model_path)


np.set_printoptions(precision=0)

## Preprocess data


In [11]:
(mnist_digits, mnist_labels), (test_digits, test_labels) = keras.datasets.mnist.load_data()
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
test_digits = np.expand_dims(test_digits, -1).astype("float32") / 255
input_shape = mnist_digits.shape[1:]
num_digits = len(mnist_labels)
sample_index = rnd.randint(0, num_digits, size = (num_samples,))
mnist_digits = mnist_digits[sample_index]
mnist_labels = mnist_labels[sample_index]

## Load Variational autoencoder

In [13]:
models = []
for i in range(num_models):
    if model_type[i] == "ae":
        models.append(keras.models.load_model(model_path[i], 
                         custom_objects={"Autoencoder": Autoencoder,
                                         "cont_bern_loss":  cont_bern_loss}))
    elif model_type[i] == "vae":
        models.append(keras.models.load_model(model_path[i], 
                                 custom_objects={"VariationalAutoencoder": VariationalAutoencoder,
                                                 "cont_bern_loss":  cont_bern_loss})) 

In [14]:
models

### Print setup details

In [16]:
print("Input shape: ", input_shape)
print("Latent dim: ",  models[0].latent_dim)


Input shape:  (28, 28, 1)
Latent dim:  10


## Reconstruct and dimension reductions

In [18]:
# Reconstructions
  = []
digit_encoded = []
digit_log_var = []

for i in range(num_models):
    if model_type[i] == "ae":
        digit_encoded.append(models[i].encoder(mnist_digits).numpy())
        digit_distribution.append(None)
        digit_log_var.append(None)
    elif model_type[i] == "vae":
        digit_distribution.append(models[i].encoder(mnist_digits).numpy())
        digit_encoded.append(digit_distribution[i][:,0,:])
        digit_log_var.append(digit_distribution[i][:,1,:])


In [19]:
# t-SNE
digit_tsne = []
for i in range(num_models):
    digit_tsne.append(TSNE(n_components = 2).fit_transform(digit_encoded[i]))

In [21]:
#pca
latent_mean = []
latent_std = []
latent_axis = []
for i in range(num_models):
    pca = PCA().fit(digit_encoded[i])
    latent_std.append(np.sqrt(pca.explained_variance_))
    latent_mean.append(pca.mean_)
    latent_axis.append(pca.components_)


In [72]:
digit_dist = []
digit_std = []
for i in range(num_models):
    digit_dist.append(np.linalg.norm(digit_encoded[i]-latent_mean[i], axis = 1))
    if model_type[i] == "ae":
        digit_std.append(None)
    elif model_type[i] == "vae":
        digit_std.append(np.sqrt(np.exp(digit_log_var[i].flatten())))


## Investigation

### Show reconstructions

In [22]:

plt.rc('font', family ='Courier New')  


In [23]:
num_col = 4
num_row = 3
num_img = num_row*num_col
rec_index = rnd.randint(num_samples,size =(num_row*num_col,) )
digits = test_digits
digits = digits[rec_index]

In [86]:
model_i = 0
model = models[model_i]

reconstructions = model(digits)
i = 0
for col in range(num_col):
    for row in range(num_row): 
        pair = np.concatenate((digits[i], reconstructions[i]), axis = 1)
        if row == 0: 
            ver_img = pair
        else:
            ver_img = np.concatenate((ver_img,
                                 pair), axis = 0)
        i = i + 1
    if col == 0:
        img = ver_img
    else: 
        img = np.concatenate((img,
                              ver_img), axis = 1)

sns.heatmap(img[:,:,0], vmin=0, vmax = 1)

<AxesSubplot:>

In [46]:
num_dig = 14
rec_index = rnd.randint(num_samples,size =(num_dig,) )
digits = mnist_digits
digits = digits[rec_index]

reconstructions = []
for model_i in range(num_models):
    reconstructions.append(models[model_i](digits))
    
i = 0

for dig_i in range(num_dig):
    rec_im = digits[dig_i]
    for model_i in range(num_models):
        
        rec_im = np.concatenate((rec_im,
                         reconstructions[model_i][dig_i]),
                         axis = 0)
    if dig_i == 0:
        im = rec_im
    else:
        im = np.concatenate((im, rec_im), axis = 1)
    
sns.heatmap(img[:,:,0], vmin=0, vmax = 1)

<AxesSubplot:>

In [119]:
x_train = mnist_digits
indeces = [1,2,3]
im = x_train[indeces[0:1],:]
im.shape

(1, 28, 28, 1)

### Latent

In [108]:

indeces = rnd.randint(num_samples,size = (3,))
indeces = [1,2,3]
ul = mnist_digits[indeces[0]]
ur = mnist_digits[indeces[1]]
dl = mnist_digits[indeces[2]]
z =  np.zeros(dl.shape)
corner_image = np.concatenate( (np.concatenate((ul,ur),axis = 1),
                                np.concatenate((dl,z ),axis = 1)), axis = 0)


In [109]:
model_i = 0
model = models[model_i]
mosaic = LatentInterpolationMosaic(
                          model.encode,
                          model.decoder,
                          mnist_digits,
                          indeces,
                          num_row = 15,
                          num_col = 15).mosaic



sns.heatmap(mosaic[:,:,0], vmin = 0, vmax = 1)

<AxesSubplot:>

### Scatter

In [35]:
model_i = 2
scatter = digit_tsne[model_i]
SampleScatterGUI(scatter, mnist_labels, mnist_digits)


In [100]:
models[1].sampler


In [50]:
model_i = 2
scatter = digit_tsne[model_i]
sns.scatterplot(x = scatter[:,0], y = scatter[:,1], hue = mnist_labels)

<AxesSubplot:>

### Spread

In [107]:
a = np.array([1,2,3])
a[0:1]

array([1])

In [61]:
sns.lineplot(data = latent_std)

<AxesSubplot:>

[array([ 8.,  9.,  8., ..., 10., 10.,  6.], dtype=float32),
 array([1., 0., 1., ..., 2., 1., 2.], dtype=float32),
 array([2., 2., 2., ..., 2., 3., 2.], dtype=float32),
 array([3., 3., 3., ..., 4., 4., 2.], dtype=float32)]

Here we have that the first 6 principal axises are almost one and the remaining 4 are almost zero (e-3). This is combination with that the cross covariance seem to be super much zero (e-15), indicates that we have a six dimesnioal sphere in this ten dimesional latent space. Interesting to see is also if you create a latent vector in the pca-vector space. Then comparing the latent value in each dimension times the spread in each dimension, gives you the importance of that dimension in that latent point. A low importance will not have an effect on the output (kind of like a low derivative...), while a hight value will give a big difference.

In [63]:
axis_index = [0,-1]
scaling_factors = np.array([5,5])

In [67]:
model_i = 1
model = models[model_i]
latent_vectors = latent_axis[model_i][axis_index]
latent_origin = latent_mean[model_i]

mosaic = LatentPlaneMosaic(model.decoder,
                  latent_vectors = (scaling_factors*latent_vectors.T).T,
                  latent_origin = latent_origin,
                  num_row = 20,
                  num_col = 20).mosaic

#fig,ax = plt.subplots(1,1)
#ax.imshow(mosaic)
sns.heatmap(mosaic[:,:,0], vmin = 0, vmax = 1)

<AxesSubplot:>

## Spred 

In [85]:
g = sns.FacetGrid( data = df, col = "digit_dist")
g.map(sns.displot)

TypeError: unhashable type: 'numpy.ndarray'

In [76]:
sns.displot(digit_dist[1])

In [264]:

fig, axs = plt.subplots(3,4)
axs[0,0].hist(dist_0, nbins, density = False)
axs[2,0].plot(density_x_0, density_0)


axs[0,1].hist(dist_1, nbins, density = False)
axs[1,1].hist(std_1, nbins,range = (0,1.1), density = True)
axs[2,1].plot(density_x_1, density_1)

axs[0,2].hist(dist_2, nbins, density = False)
axs[1,2].hist(std_2, nbins,range = (0,1.1), density = False)
axs[2,2].plot(density_x_2, density_2)

axs[0,3].hist(dist_3, nbins, density = False)
axs[1,3].hist(std_3, nbins,range = (0,1.1), density = False)
axs[2,3].plot(density_x_3, density_3)

## Generative power

In [265]:
mean = np.zeros(latent_dim)
mean = latent_mean_0
cov  = 1*np.eye(latent_dim)
num_col = 15
num_row = 10
num_img = num_row*num_col
latent_points = rnd.multivariate_normal(mean, cov, size = (num_img))



In [218]:
model = model_0
title = "Reconstruction with autoencoder"
fig, ax = plt.subplots()
reconstructions = model.decoder(latent_points)
i = 0
for col in range(num_col):
    for row in range(num_row): 
        if row == 0: 
            ver_img = reconstructions[i]
        else:
            ver_img = np.concatenate((ver_img,
                                 reconstructions[i]), axis = 0)
        i = i + 1
    if col == 0:
        img = ver_img
    else: 
        img = np.concatenate((img,
                              ver_img), axis = 1)
ax.imshow(img)
ax.set_title(title)


Text(0.5, 1.0, 'Reconstruction with autoencoder')

In [266]:
title = "Progression of models (Org, AE, AE_cb, VAE100,VAE100_cb)"
fig, axs = plt.subplots(num_row, num_col)
rec0 = model_0.decoder(latent_points)
rec1 = model_1.decoder(latent_points)
rec2 = model_2.decoder(latent_points)
rec3 = model_3.decoder(latent_points)

i = 0
for col in range(num_col):
    for row in range(num_row): 
        ax = axs[row, col]
        im = np.concatenate((rec0[i],
                             rec1[i],
                             rec2[i],
                             rec3[i]),
                             axis = 1)
        ax.imshow(im)
        ax.set_title(title)
        i = i +1

In [274]:
mean = np.zeros(latent_dim)
cov  = 1*np.eye(latent_dim)
num_dig = 14
latent_points = rnd.multivariate_normal(mean, cov, size = (num_dig))

fig, ax = plt.subplots()
title = "Progression of models (Org, AE, AE_cb, VAE100,VAE100_cb)"
rec0 = model_0.decoder(latent_points)
rec1 = model_1.decoder(latent_points)
rec2 = model_2.decoder(latent_points)
rec3 = model_3.decoder(latent_points)


i = 0
for i in range(num_dig): 
    rec_im = np.concatenate((rec0[i],
                         rec1[i],
                         rec2[i],
                         rec3[i]),
                         axis = 0)
    if i == 0:
        im = rec_im
    else:
        im = np.concatenate((im, rec_im), axis=1)
    i = i+1
    
ax.imshow(im)
ax.set_title(title)

Text(0.5, 1.0, 'Progression of models (Org, AE, AE_cb, VAE100,VAE100_cb)')

In [82]:
import pandas as pd


In [83]:
df = pd.DataFrame({
    "digit_dist" : digit_dist,
    "digit_std" : digit_std
})

In [84]:
df.head()

,digit_dist,digit_std
0,"[8.222564, 9.192295, 7.5260377, 7.5173655, 11....",None
1,"[0.66744155, 0.2013679, 1.3767942, 1.0412824, ...","[0.35392433, 0.2852933, 1.0034171, 1.0030283, ..."
2,"[2.1573875, 2.1830826, 2.2218325, 2.290831, 2....","[0.8652521, 0.042920884, 0.099526055, 0.103524..."
3,"[2.886057, 2.5140355, 2.768398, 2.8142986, 4.2...","[0.018387135, 0.024138493, 0.02755069, 0.02764..."
